In [2]:
T = "alabar_a_la_alabarda$"

In [81]:
def prefix_suffix_match(prefix, suffix, debug=True):
    if debug:
        print("Checking {} and {}".format(suffix, prefix))
    if len(prefix) < len(suffix):
        for i in range(len(prefix)):
            prefixlen = len(prefix) - i
            if suffix[len(suffix)-prefixlen:len(suffix)] == prefix[0:prefixlen]:
                if debug:
                    print("Found match {}".format(prefix[0:prefixlen]))
                return prefix[0:prefixlen]
            else: 
                if debug:
                    print(suffix[len(suffix)-prefixlen:len(suffix)], prefix[0:prefixlen])
    else: # case len(prefix) >= len(suffix)
        for i in range(len(suffix)):
            suffixlen = len(suffix) - i
            if suffix[len(suffix)-suffixlen:len(suffix)] == prefix[0:suffixlen]:
                if debug:
                    print("Found match {}".format(prefix[0:suffixlen]))
                return prefix[0:suffixlen]
            else:
                if debug:
                    print(suffix[len(suffix)-suffixlen:len(suffix)], prefix[0:suffixlen])
    return ''
        

In [210]:
def LZ_End_parser(text, debug=True):
    print('Parsing text {}'.format(text))
    sourcetext = [] 
    source = [0] # phrase id of source, source index starts from 0
    B = [] # bitmap marks the ending of a phrase, index start from 0
    L = [''] # trailing characters start from 0
    textpos = 0
    phraseid = 0
    while len(text) > 0:
        # sourcetext.append(''.join(source))
        if debug: 
            for oneline in sourcetext:
                print(oneline)
        maxlen = 0
        maxtext = ''
        q = 0
        for id, oneline in enumerate(sourcetext):
            # find the prefix of text that is suffix of oneline 
            match = prefix_suffix_match(text, oneline, False)
            if len(match) > maxlen:
                maxlen = len(match)
                maxtext = match
                q = id + 1
        # maxtext now is the new phrase to add 
        if debug:
            print("Match {} with length {}".format(maxtext, len(maxtext)))
            
        B.extend([0]*len(maxtext))
        B.append(1)
        L.append(text[len(maxtext)])
        source.append(q)
        
        phrase = text[0:len(maxtext)+1]
        text = text[len(maxtext)+1:len(text)]
        print("New phrase {}".format(phrase))
        if debug:
            print(text)
        if  len(sourcetext) > 0:
            sourcetext.append(''.join([sourcetext[-1], phrase]))
        else:
            sourcetext.append(phrase)
    return source, B, L


In [211]:
source, B, L = LZ_End_parser(T, False)
print(source)
print(B)
print(L)

Parsing text alabar_a_la_alabarda$
New phrase a
New phrase l
New phrase ab
New phrase ar
New phrase _
New phrase a_
New phrase la
New phrase _a
New phrase labard
New phrase a$
[0, 0, 0, 1, 1, 0, 1, 2, 5, 4, 1]
[1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1]
['', 'a', 'l', 'b', 'r', '_', '_', 'a', 'a', 'd', '$']


In [212]:
def rank(b, B, pos):
    count = 0
    for i in range(pos+1):
        if B[i] == b:
            count += 1
    return count

def select(b, B, th):
    count = 0
    for i in range(len(B)):
        if B[i] == b: 
            count += 1
        if count == th:
            return i
    return 0

In [213]:
def extract(start, l, debug = True):
    print("\n")
    print("Extracting from {} with len {}".format(start, l))
    if l > 0:
        print("start = {}, l = {}".format(start, l))
        end = start + l - 1 
        p = rank(1, B, end - 1) + 1
        if debug:
            print("p = {}".format(p))
        if B[end]:
            print("End of phrase {} call extract({},{}) from extract({},{})".format(p, start, l-1, start, l))
            # print("Call extract({},{})".format(start, l-1))
            extract(start, l-1)
            print ("OUTPUT from {} len {} is trailing of phrase {} : {}".format( start, l, p, L[p])) #coded source start with 0, real source starts with 1
        else:
            pos = select(1, B, p-1) + 1
            if debug:
                print("pos = {}".format(pos)) 
            oldstart = start 
            oldl = l
            if start < pos: 
                if debug:
                    print("Call extract({},{}) from extract({},{})".format(start, pos - start, start, l))
                extract(start, pos-start)
                l = end-pos + 1
                start = pos 
                
            if debug:
                print("source of {} is {}".format(p, source[p])) #coded source start with 0, real source starts with 1
                print("Pos of the ending of the {} phrase is {}".format(source[p]-1, select(1, B, source[p] - 1)))
                print("Not end of phrase, call extract({},{}) from extract({},{})".
                      format(select(1, B, source[p] - 1) + start - pos + 1, l, oldstart, oldl))
            extract(select(1, B, source[p] - 1) + start - pos + 1, l)
    
        

In [214]:
extract(11, 5)



Extracting from 11 with len 5
start = 11, l = 5
p = 9
pos = 13
Call extract(11,2) from extract(11,5)


Extracting from 11 with len 2
start = 11, l = 2
p = 8
End of phrase 8 call extract(11,1) from extract(11,2)


Extracting from 11 with len 1
start = 11, l = 1
p = 8
pos = 11
source of 8 is 5
Pos of the ending of the 4 phrase is 5
Not end of phrase, call extract(6,1) from extract(11,1)


Extracting from 6 with len 1
start = 6, l = 1
p = 5
End of phrase 5 call extract(6,0) from extract(6,1)


Extracting from 6 with len 0
OUTPUT from 6 len 1 is trailing of phrase 5 : _
OUTPUT from 11 len 2 is trailing of phrase 8 : a
source of 9 is 4
Pos of the ending of the 3 phrase is 3
Not end of phrase, call extract(4,3) from extract(11,5)


Extracting from 4 with len 3
start = 4, l = 3
p = 5
End of phrase 5 call extract(4,2) from extract(4,3)


Extracting from 4 with len 2
start = 4, l = 2
p = 4
End of phrase 4 call extract(4,1) from extract(4,2)


Extracting from 4 with len 1
start = 4, l = 1
p = 